## Model Tuning for Note Transcription

In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow import keras
import keras_tuner as kt
from keras_tuner.tuners import Hyperband
from build_train_set import buildTrainSet
from post_transcription import returnPvLabels
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
x_train, y_train, label_train, x_test, y_test, label_test, x_tune, y_tune, label_tune = buildTrainSet()
print((x_train.shape, y_train.shape), (x_test.shape, y_test.shape), (x_tune.shape, y_tune.shape))

((3638, 250, 1), (3638, 250, 13)) ((396, 250, 1), (396, 250, 13)) ((396, 250, 1), (396, 250, 13))


In [3]:
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('input_unit',min_value=32,max_value=512,step=32),
                    return_sequences=True,
                    input_shape=(x_train.shape[1], x_train.shape[2])))
    for i in range(hp.Int('n_layers',1,4)):
        model.add(LSTM(units=hp.Int('lstm_{i}_units',min_value=32,max_value=512,step=32),return_sequences=True))
    model.add(Dropout(hp.Float('dropout_rate',min_value=0,max_value=0.5,step=0.1)))
    model.add(Dense(13, activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3, 1e-4])),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

    return model
        

In [4]:
tuner = Hyperband(
            hypermodel=build_model,
            objective='val_accuracy',
            max_epochs=10,
            factor=3,
            project_name='lstm_tuner')

In [5]:
tuner.search(x_train, y_train, epochs=5, batch_size=256, validation_data=(x_test,y_test))

Trial 30 Complete [00h 30m 14s]
val_accuracy: 0.6432828307151794

Best val_accuracy So Far: 0.7879495024681091
Total elapsed time: 13h 01m 01s
INFO:tensorflow:Oracle triggered exit


In [7]:
tuner.results_summary()

Results summary
Results in .\lstm_tuner
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0028 summary
Hyperparameters:
input_unit: 352
n_layers: 3
lstm_{i}_units: 480
dropout_rate: 0.2
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.7879495024681091

Trial 0027 summary
Hyperparameters:
input_unit: 352
n_layers: 2
lstm_{i}_units: 512
dropout_rate: 0.30000000000000004
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.7625757455825806

Trial 0026 summary
Hyperparameters:
input_unit: 352
n_layers: 3
lstm_{i}_units: 384
dropout_rate: 0.2
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.7494545578956604

Trial 0017 summary
Hyperparameters:
input_unit: 256
n_layers: 1
lstm_{i}_units: 128
dropout_rate: 0.1
learning_rate: 0.01
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_

## Model Training and Checking for Number of Epochs

In [9]:
model = Sequential()
model.add(LSTM(units=352, return_sequences=True, input_shape=(250,1)))
model.add(LSTM(units=480, return_sequences=True))
model.add(LSTM(units=480, return_sequences=True))
model.add(LSTM(units=480, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(Dense(13, activation='softmax'))

opt = keras.optimizers.Adam(learning_rate=0.001) 
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 250, 352)          498432    
                                                                 
 lstm_5 (LSTM)               (None, 250, 480)          1599360   
                                                                 
 lstm_6 (LSTM)               (None, 250, 480)          1845120   
                                                                 
 lstm_7 (LSTM)               (None, 250, 480)          1845120   
                                                                 
 dropout_1 (Dropout)         (None, 250, 480)          0         
                                                                 
 dense_1 (Dense)             (None, 250, 13)           6253      
                                                                 
Total params: 5,794,285
Trainable params: 5,794,285
No

In [ ]:
hist = model.fit(x_train, y_train, epochs=200, batch_size=250, validation_data=(x_test, y_test))

model.save('model_lstm.h5') # backup